In [0]:
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
# import library 
import os
import numpy as np
import pandas as pd
from skimage import io, transform

import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.models as models
import torch.nn as nn 

In [0]:
#import 
import PIL
print(PIL.PILLOW_VERSION)

4.1.1


In [0]:
# install pillow 4.1.1 version
!pip install Pillow==4.1.1



    100% |████████████████████████████████| 5.7MB 7.7MB/s 
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0


In [0]:
from PIL import Image

In [0]:
#files upload to google drive
#import drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
print("GPU is available" if torch.cuda.is_available() else "GPU is not available")

GPU is available


In [0]:
#csv_file='/content/gdrive/My Drive/original_test.csv'
root_dir='/content/gdrive/My Drive/train_corrected/train'


**please change test csv file path and directory path**

In [0]:
#import test data
# target to numbers 
test_data=pd.read_csv(csv_file) # file path of original test data
img_name=test_data['image_name']
target=test_data['target']

target = target.map({
    "bus": 0,
    "cement mixer": 1,
    "crane truck": 2,
    "dedicated agricultural vehicle": 3,
    "hatchback": 4,
    "jeep": 5,
    "light truck": 6,
    "minibus": 7,
    "minivan": 8,
    "pickup": 9,
    "prime mover": 10,
    "sedan": 11,
    "tanker": 12,
    "truck": 13,
    "van": 14 })

#test_data.to_csv('/content/gdrive/My Drive/new_test')
#print(test_data)
new_test = pd.concat([img_name, target], axis = 1, join_axes = [img_name.index])
new_test.to_csv(path_or_buf = '/content/gdrive/My Drive/new_test.csv', index = False)
#if need index 
#new_test.to_csv(path_or_buf='/content/gdrive/My Drive/new_test.csv',index=True)

In [0]:
#bus: 0
#cement mixer:1 
#crane truck: 2
#dedicated agricultural vehicle
#hatchback: 4
#jeep: 5
#minibus: 7
#minivan: 8
#pickup: 9
#prime mover: 10
#sedan: 11
#tanker: 12
#truck: 13
#van:14


In [0]:
#load csv file after mapping
csv_file='/content/gdrive/My Drive/new_test.csv'

**please change csv file path and root dir**

#loading test file

In [0]:
#dataset
class CarDataset(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
    # csv file path
    # root dir
    self.target_frame = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform
  
  def __len__(self):
    return len(self.target_frame)

  def __getitem__(self, idx):
    img_name = os.path.join(self.root_dir,
                            self.target_frame.iloc[idx,0]) # idx 0: image_name
    image = Image.open(img_name)
    target=self.target_frame.iloc[idx,1] # idx 1: target
    #target=int(target)
    #sample = {'image': image}
    if self.transform:
            image = self.transform(image)

    return image,target


**note: if using index is set True at previous step, [idx,0] change to [idx,1], [idx,1] change to [idx,2]**

In [0]:
#normalize by mean and std 
normalize=transforms.Normalize(mean=[126.387,131.316,144.899],
                              std=[36.263,37.283,36.225])
test_dataset = CarDataset(csv_file=csv_file,    # csv file after mapping
                                           root_dir=root_dir,
                                           transform=transforms.Compose([
                                               #transforms.ToPILImage(),
                                               transforms.Resize((200,200)),
                                               transforms.Pad(12,fill=0,padding_mode='constant'),  
                                               #normalize,
                                               transforms.ToTensor(),
                                               normalize        
                                           ]))

In [0]:
test_dataloader=DataLoader(test_dataset,batch_size=32)

In [0]:
print(len(test_dataloader.dataset))

581


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
# test on test set 
def test(model,dataloader): 
    model.eval()
    running_loss = 0
    running_corrects = 0
    with torch.no_grad():
      for i, (inputs,labels) in enumerate (dataloader):
        inputs = torch.tensor(inputs, dtype=torch.float, device=device)
        labels = torch.tensor(labels, dtype=torch.long, device=device)
        #print(inputs)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels).item()
        running_loss = running_loss / len(dataloader.dataset) #loss
        running_acc = running_corrects / len(dataloader.dataset) #accuracy
    
    print("test acc::{:.6f}".format(running_acc))
    
  

#loading model

change the file path of a model 

In [0]:
#load one of previously training models
#original_thu is only a test
test_model = models.resnet18(pretrained = True)  #resnet18
num_ftrs=test_model.fc.in_features
test_model.fc=nn.Linear(num_ftrs,15) #number of features is 15
test_model.load_state_dict(torch.load('/content/gdrive/My Drive/train_corrected/original_thu.pt')) # file path of one model
test_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [0]:
criterion = nn.CrossEntropyLoss()
test=test(test_model,test_dataloader)

test acc::0.703959


In [0]:
#Merry Christmas and Happy New Year!